<a href="https://colab.research.google.com/github/Shravikajain/Ai-S/blob/master/transferlearning_scaling__up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-07-02 06:18:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-07-02 06:18:41 (102 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback , plot_loss_curves , unzip_data ,compare_historys , walk_through_dir

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2024-07-02 06:18:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.207, 173.194.210.207, 173.194.212.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  39.4MB/s    in 21s     

2024-07-02 06:19:13 (72.2 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [ ]:
walk_through_dir("10_food_classes_10_percent")

In [ ]:
import tensorflow as tf
IMG_SIZE = (224,224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                               label_mode = "categorical",
                                                                               image_size = IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode = "categorical",
                                                                image_size = IMG_SIZE,
                                                                shuffle = False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [ ]:
checkpoint_path = "101_classes_1-_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only = True,
                                                         monitor = "val_accuracy",
                                                         save_best_only = True)



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomHeight(0.1),
        layers.RandomWidth(0.1),
        # layers.experimental.preprocessing.Rescaling(1./255)
    ],name = "data_augmentation"
)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

inputs = layers.Input(shape = (224,224,3), name = "input_layer")
x = data_augmentation(inputs)
x = base_model(x, training = False)
x = layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
outputs = layers.Dense(101, activation = "softmax", name = "output_layer")(x)
model = tf.keras.Model(inputs, outputs)


16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
#  model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs =5,
                                           validation_data = test_data,
                                           validation_steps = int(0.15*len(test_data)),
                                           callbacks = [checkpoint_callback])

Epoch 1/5
237/237 [==============================] - 959s 4s/step - loss: 3.2755 - accuracy: 0.2990 - val_loss: 2.4157 - val_accuracy: 0.4650
Epoch 2/5
237/237 [==============================] - 962s 4s/step - loss: 2.0409 - accuracy: 0.5405 - val_loss: 1.9630 - val_accuracy: 0.5275
Epoch 3/5
237/237 [==============================] - 954s 4s/step - loss: 1.6500 - accuracy: 0.6147 - val_loss: 1.8431 - val_accuracy: 0.5355
Epoch 4/5
237/237 [==============================] - 958s 4s/step - loss: 1.4172 - accuracy: 0.6640 - val_loss: 1.7636 - val_accuracy: 0.5456
Epoch 5/5
214/237 [==========================>...] - ETA: 1:02 - loss: 1.2495 - accuracy: 0.7037

In [ ]:
feature_extraction_results = model.evaluate(test_data)
print(feature_extraction_results)

790/790 [==============================] - 2014s 3s/step - loss: 1.5359 - accuracy: 0.5926
[1.5358779430389404, 0.5925940871238708]


In [ ]:
# plot_loss_curves(history_all_classes_10_percent)

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-5]:
  layer.trainable = False



In [ ]:
model.compile(loss = "categorical_crossentropy",
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
              metrics = ["accuracy"])

In [ ]:
# for layer in model.layers:
  # print(layer.name , layer.trainable)

In [ ]:
# for layer_number ,layer in enumerate(model.layers[2].layers):
  # print(layer_number, layer.name , layer.trainable)


In [ ]:
fine_tune_epochs = 10
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs = fine_tune_epochs,
                                                     validation_data = test_data,
                                                     validation_steps = int(0.15*len(test_data)),
                                                     initial_epoch = history_all_classes_10_percent.epoch[-1]
                                                     )

Epoch 5/10
237/237 [==============================] - 915s 4s/step - loss: 0.5537 - accuracy: 0.8622 - val_loss: 1.7148 - val_accuracy: 0.5604
Epoch 6/10
237/237 [==============================] - 1034s 4s/step - loss: 0.5042 - accuracy: 0.8742 - val_loss: 1.7107 - val_accuracy: 0.5654
Epoch 7/10
116/237 [=============>................] - ETA: 5:15 - loss: 0.4823 - accuracy: 0.8831

KeyboardInterrupt: 

In [ ]:
all_classes_10_percent_fine_tune_results = model.evalute(test_data)
print(all_classes_10_percent_fine_tune_results)

AttributeError: 'Functional' object has no attribute 'evalute'

In [ ]:
campare_historys(original_history = history_all_classes_10_percent,
                 new_history = history_all_classes_10_percent_fine_tune,
                 initial_epochs = 5)


In [ ]:
model.save("drive/MyDrive/tensorflow_course/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
loaded_model = tf.keras.load_model("drive/MyDrive/tensorflow_course/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
loaded_model_results = loaded_model.evaluate(test_data)
loaded_model_results

In [ ]:
all_classes_10_percent_fine_tune_results

In [ ]:
import tensorflow as tf

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [ ]:
unzip_data("/content/06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
model.load_model = tf.kerasdel("/content/06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
results_downloaded_model = model.evaluate(test_data)
results_downloaded_model

In [ ]:
preds_probs = model.predict(test_data,verbose = 1)


In [ ]:
preds_probs[0] , len(preds_probs[0]), sum(preds_probs[0])

In [ ]:
import matplotlib.pyplot as plt
fig , ax = plt.subplot(figsize =(12,25))
scores = ax.barh(range )